In [1]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, load_dataset, interleave_datasets, load_from_disk
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer, pipeline
import torch
import time
import evaluate
import pandas as pd
import numpy as np
from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_int8_training, TaskType

2025-03-21 15:15:14.814900: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-21 15:15:14.814953: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-21 15:15:14.816575: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-21 15:15:14.824509: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-21 15:15:16.298606: W tensorflow/compiler/tf2

In [4]:
model_name='t5-small'

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
model = model.to('cuda')

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# Load data

Combining 3 datasets by renaming the columns and then split into train-val-test set by 8:1:1

In [ ]:
try:
    dataset = load_from_disk("merged_dataset")
    print("Loaded Merged Dataset")
except:
    dataset_scc_train = load_dataset("b-mc2/sql-create-context", split='train[:80%]')
    dataset_scc_test  = load_dataset("b-mc2/sql-create-context", split='train[-20%:-10%]')
    dataset_scc_val   = load_dataset("b-mc2/sql-create-context", split='train[-10%:]')

    dataset_tts_train = load_dataset("Clinton/Text-to-sql-v1", split='train[:80%]')
    dataset_tts_train = dataset_tts_train.remove_columns(['source', 'text'])
    dataset_tts_train = dataset_tts_train.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'})
    dataset_tts_test  = load_dataset("Clinton/Text-to-sql-v1", split='train[-20%:-10%]')
    dataset_tts_test  = dataset_tts_test.remove_columns(['source', 'text'])
    dataset_tts_test  = dataset_tts_test.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'})
    dataset_tts_val   = load_dataset("Clinton/Text-to-sql-v1", split='train[-10%:]')
    dataset_tts_val   = dataset_tts_val.remove_columns(['source', 'text'])
    dataset_tts_val   = dataset_tts_val.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'})

    dataset_ks_train  = load_dataset("knowrohit07/know_sql", split='validation[:80%]')
    dataset_ks_test   = load_dataset("knowrohit07/know_sql", split='validation[-20%:-10%]')
    dataset_ks_val    = load_dataset("knowrohit07/know_sql", split='validation[-10%:]')

    dataset = DatasetDict({ 'train': interleave_datasets([dataset_scc_train, dataset_tts_train, dataset_ks_train]),
                            'test': interleave_datasets([dataset_scc_test, dataset_tts_test, dataset_ks_test]),
                            'validation': interleave_datasets([dataset_scc_val, dataset_tts_val, dataset_ks_val])})

    dataset.save_to_disk("merged_dataset")
    print("Merged and Saved Dataset")

dataset

# Preprocess the Datasets

Tokenization + padding

In [5]:
def tokenize_function(example):
    start_prompt = "Tables:\n"
    middle_prompt = "\n\nQuestion:\n"
    end_prompt = "\n\nAnswer:\n"
  
    data_zip = zip(example['context'], example['question'])
    prompt = [start_prompt + context + middle_prompt + question + end_prompt for context, question in data_zip]

    # Tokenize input (context + question)
    inputs = tokenizer(
        prompt, 
        return_tensors="pt",
        truncation=True,
        padding = True
    )

    # Tokenize output (answer)
    labels = tokenizer(
        example['answer'], 
        return_tensors="pt",
        truncation=True,
        padding = True
    )

    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": labels["input_ids"]
    }

In [6]:
try:
    tokenized_datasets = load_from_disk("tokenized_datasets")
    print("Loaded Tokenized Dataset")
except:
    tokenized_datasets = dataset.map(
                                    tokenize_function,
                                    batched=True,  
                                    remove_columns=['question', 'context', 'answer'],  # Remove original text fields
                                    load_from_cache_file=False,
                                    desc="Tokenizing dataset",
                                )
    
    tokenized_datasets.save_to_disk("tokenized_datasets")
    print("Tokenized and Saved Dataset")

Loaded Tokenized Dataset


In [4]:
# print(tokenized_datasets.keys())
# print(tokenized_datasets['train'][0].keys())
# print(tokenized_datasets['train'][0]['input_ids'])
# print(tokenized_datasets['train'][0]['labels'])
# print(tokenized_datasets)

# Configure LoRA adapter 

In [11]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    # target_modules=["q", "v", "o", "wi_0", "wi_1", "wo"],
    # modules_to_save=["lm_head", "embed_tokens"],
    task_type=TaskType.SEQ_2_SEQ_LM,
)

In [12]:
model_name='t5-small'

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
model = model.to('cuda')

In [13]:
model = prepare_model_for_int8_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 294,912 || all params: 60,801,536 || trainable%: 0.4850403779272945


/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:136: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


# Finetuning LoRA adapter

In [15]:
from transformers import DataCollatorForSeq2Seq
label_pad_token_id = -100
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [16]:
output_dir = f'tunes/sql-training-{str(int(time.time()))}'
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=5e-3,
    num_train_epochs=2,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=500,
    per_device_train_batch_size=16,     # batch size per device during training
    per_device_eval_batch_size=16,      # batch size for evaluation
    weight_decay=0.01,
    save_strategy="steps",
    label_names = ['labels']
)

In [13]:
tokenized_datasets['train']

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 118695
})

In [17]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)

[2025-03-21 00:11:49,383] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [18]:
model.config.use_cache = False

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: kookyghost (kookyghost-university-of-connecticut). Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.189200
1000,0.079800
1500,0.072200
2000,0.066400
2500,0.061800
3000,0.061300
3500,0.055800
4000,0.052900
4500,0.057000
5000,0.057900


TrainOutput(global_step=14838, training_loss=0.05259703612902814, metrics={'train_runtime': 8471.1663, 'train_samples_per_second': 28.023, 'train_steps_per_second': 1.752, 'total_flos': 3.234385838997504e+16, 'train_loss': 0.05259703612902814, 'epoch': 2.0})

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f841a629410>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f841a730f90, execution_count=18 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f8419af2fd0, raw_cell="model.config.use_cache = False

trainer.train()" store_history=True silent=False shell_futures=True cell_id=bc0c79b9-1e99-4c31-97b6-47d998eba6b9> result=TrainOutput(global_step=14838, training_loss=0.05259703612902814, metrics={'train_runtime': 8471.1663, 'train_samples_per_second': 28.023, 'train_steps_per_second': 1.752, 'total_flos': 3.234385838997504e+16, 'train_loss': 0.05259703612902814, 'epoch': 2.0})>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

# Save Trained Model

In [20]:
trainer.model.save_pretrained('tuned_model')
tokenizer.save_pretrained('tuned_model')

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f841a629410>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f83bc691310, raw_cell="trainer.model.save_pretrained('tuned_model')
token.." store_history=True silent=False shell_futures=True cell_id=ea600991-24a2-47aa-8ac3-a1c3673c5b5f>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

('tuned_model/tokenizer_config.json',
 'tuned_model/special_tokens_map.json',
 'tuned_model/spiece.model',
 'tuned_model/added_tokens.json',
 'tuned_model/tokenizer.json')

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f841a629410>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f83bc406950, execution_count=20 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f83bc691310, raw_cell="trainer.model.save_pretrained('tuned_model')
token.." store_history=True silent=False shell_futures=True cell_id=ea600991-24a2-47aa-8ac3-a1c3673c5b5f> result=('tuned_model/tokenizer_config.json', 'tuned_model/special_tokens_map.json', 'tuned_model/spiece.model', 'tuned_model/added_tokens.json', 'tuned_model/tokenizer.json')>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

# Compare Parameter Numbers between Base Model and LoRA Adapter

In [17]:
import os

In [5]:
def print_model_size(path):
    size = 0
    for f in os.scandir(path):
        size += os.path.getsize(f)

    print(f"Model size: {(size / 1e6):.2} MB")

In [6]:
def print_trainable_parameters(model, label):
    parameters, trainable = 0, 0
    
    for _, p in model.named_parameters():
        parameters += p.numel()
        trainable += p.numel() if p.requires_grad else 0

    print(f"{label} trainable parameters: {trainable:,}/{parameters:,} ({100 * trainable / parameters:.2f}%)")


In [7]:
model_name='t5-small'

tokenizer = AutoTokenizer.from_pretrained(model_name)

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
original_model = original_model.to('cuda')

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [8]:
print_trainable_parameters(original_model,'base model')

base model trainable parameters: 60,506,624/60,506,624 (100.00%)


In [14]:
model = get_peft_model(original_model, lora_config)

In [15]:
print_trainable_parameters(model,'LoRA model')

LoRA model trainable parameters: 294,912/60,801,536 (0.49%)


In [18]:
print_model_size('tuned_model')

Model size: 4.4 MB


In [25]:
trainer.save_model('tuned_model_save_model')

In [21]:
import wandb
wandb.finish()
# wandb.login()  

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f841a629410>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f83d2246b90, raw_cell="import wandb
wandb.finish()
# wandb.login()  " store_history=True silent=False shell_futures=True cell_id=8889f92d-469d-4a26-b9b7-db3d9651d9e8>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


train/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train/grad_norm,▆▅▄▅▃▂█▃▇▄▃▂▁▂▃▂▃▂▁▄▆▂▂▃▁▂▂▂▂
train/learning_rate,██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁▁
train/loss,█▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁
total_flos,3.234385838997504e+16
train/epoch,2.0
train/global_step,14838
train/grad_norm,0.0454
train/learning_rate,0.00011
train/loss,0.0322
